In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf

from keras.datasets import mnist
from tensorflow.keras.optimizers.legacy import SGD, Adam
from tensorflow.keras.utils import to_categorical
from keras.backend import image_data_format
from keras.applications.mobilenet import MobileNet
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from tensorflow import keras
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Dense
from keras.models import Sequential
import random
import time
import matplotlib.pyplot as plt
import numpy as np
import copy

import csv
import random
import time

# client config
NUMOFCLIENTS = 10 # number of client(as particles)
EPOCHS = 30 # number of total iteration
CLIENT_EPOCHS = 1 # number of each client's iteration
BATCH_SIZE = 5 # Size of batches to train on
ACC = 0.3 # 0.4
LOCAL_ACC = 0.7 # 0.6
GLOBAL_ACC = 1.4 # 1.0

DROP_RATE = 0 # 0 ~ 1.0 float value


# model config
LOSS = 'categorical_crossentropy' # Loss function
NUMOFCLASSES = 10 # Number of classes
lr = 0.0025
OPTIMIZER = SGD(lr=lr, momentum=0.9, decay=lr/(EPOCHS*CLIENT_EPOCHS), nesterov=False) # lr = 0.015, 67 ~ 69%



def load_dataset():
    """
    This function loads the dataset provided by Keras and pre-processes it in a form that is good to use for learning.

    Return:
        (X_train, Y_train), (X_test, Y_test)
    """

    # Code for experimenting with MNIST datasets.
    (X_train, Y_train), (X_test, Y_test) = mnist.load_data()
    X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
    X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train = X_train / 255.0
    X_test = X_test / 255.0

    Y_train = to_categorical(Y_train)
    Y_test = to_categorical(Y_test)

    return (X_train, Y_train), (X_test, Y_test)


In [ ]:
# create CNN Model

class Model():

    def __init__(self, loss, optimizer, classes=10):
        self.loss = loss
        self.optimizer = optimizer
        self.num_classes = classes

    @staticmethod
    def build(input_shape, classes, final_activation, drop_rate=0):
        model = Sequential()

        # 1
        model.add(Conv2D(
            filters=32,
            kernel_size=(5, 5),
            padding='same',
            activation='relu',
            input_shape=input_shape,
            kernel_regularizer='l2',
        ))
        model.add(Conv2D(
            filters=32,
            kernel_size=(5, 5),
            padding='same',
            activation='relu',
            kernel_regularizer='l2',
        ))
        model.add(MaxPooling2D(
            pool_size=(2,2),
            padding='same'
        ))
        model.add(Dropout(drop_rate))

        # 2
        model.add(Conv2D(
            filters=64,
            kernel_size=(5, 5),
            padding='same',
            activation='relu',
            kernel_regularizer='l2',
        ))
        model.add(Conv2D(
            filters=64,
            kernel_size=(5, 5),
            padding='same',
            activation='relu',
            kernel_regularizer='l2',
        ))
        model.add(MaxPooling2D(
            pool_size=(2,2),
            padding='same'
        ))
        model.add(Dropout(drop_rate))

        # 3
        model.add(Flatten())
        model.add(Dense(
            units=512,
            activation='relu',
            kernel_regularizer='l2',
        ))
        model.add(Dropout(drop_rate))

        # 4
        model.add(Dense(
            units=classes,
            activation=final_activation
        ))

        model.compile(
            loss='categorical_crossentropy',  # Using categorical_crossentropy since the final layer uses softmax
            optimizer=Adam(lr=0.0025),  # Adjust the learning rate if needed
            metrics=['accuracy']
        )

        return model


In [ ]:
#Client selection algorithm using knapsack model

def knapsack_with_features(clients, capacity):
    n = len(clients)
    dp = [[0 for _ in range(capacity + 1)] for _ in range(n + 1)]

    for i in range(1, n + 1):
        for w in range(capacity + 1):
            if clients[i - 1]['Energy'] <= w:
                dp[i][w] = max(dp[i - 1][w], clients[i - 1]['Reliability'] + dp[i - 1][w - clients[i - 1]['Energy']])
            else:
                dp[i][w] = dp[i - 1][w]

    selected_clients = []
    i, j = n, capacity
    while i > 0 and j > 0:
        if dp[i][j] != dp[i - 1][j]:
            selected_clients.append(i - 1)
            j -= clients[i - 1]['Energy']
        i -= 1

    selected_client_features = [clients[idx] for idx in selected_clients]

    return selected_clients, selected_client_features

# Client features:
clients = [
    {"Di": 6000, "Loss": 0.3372, "Delay": 1.421, "Energy": 58, "Reliability": 0.1210938634, "Fairness": 0.66},
    {"Di": 6000, "Loss": 0.6755, "Delay": 2, "Energy": 64, "Reliability": 0.04978706837, "Fairness": 0.83},
    {"Di": 6000, "Loss": 0.5286, "Delay": 2.16, "Energy": 47, "Reliability": 0.2493522088, "Fairness": 0.63},
    {"Di": 6000, "Loss": 0.4091, "Delay": 1.27, "Energy": 38, "Reliability": 1, "Fairness": 0.6},
    {"Di": 6000, "Loss": 0.3258, "Delay": 1.17, "Energy": 42, "Reliability": 1, "Fairness": 0.83},
    {"Di": 6000, "Loss": 0.4178, "Delay": 1.753, "Energy": 65, "Reliability": 0.1806214331, "Fairness": 1},
    {"Di": 6000, "Loss": 0.13236, "Delay": 1.8, "Energy": 36, "Reliability": 0.03567399335, "Fairness": 1},
    {"Di": 6000, "Loss": 0.2377, "Delay": 2.142, "Energy": 13, "Reliability": 1, "Fairness": 1},
    {"Di": 6000, "Loss": 0.323, "Delay": 2.52, "Energy": 16,"Reliability": 0.02811565975, "Fairness": 0.9333333333},
    {"Di": 6000, "Loss": 0.4094, "Delay": 1.8, "Energy": 96, "Reliability": 0.1888756028, "Fairness": 0.9333333333}
]
capacity = 200

selected_clients, selected_client_features = knapsack_with_features(clients, capacity)
print("Selected clients:", selected_clients)
print("Selected client features:", selected_client_features)

In [ ]:
import numpy as np
import random
import time

# Create and compile the client model
def create_client_model():
    model = Model.build(input_shape=(28, 28, 1), classes=NUMOFCLASSES, final_activation='softmax', drop_rate=DROP_RATE)
    model.compile(loss=LOSS, optimizer=OPTIMIZER, metrics=['accuracy'])
    return model

# Select clients based on knapsack scores
def select_clients(knapsack_scores, num_clients):

    selected_clients = np.argsort(knapsack_scores)[-num_clients:]
    return selected_clients

# Perform federated learning with client selection
def federated_learning_with_selection(clients, capacity, num_clients, num_epochs, client_epochs, batch_size):
    # Load dataset
    (X_train, Y_train), (X_test, Y_test) = load_dataset()

    # Initialize global model
    global_model = create_client_model()

    # Initialize knapsack scores for each client
    knapsack_scores = [random.uniform(0, 1) for _ in range(len(clients))]

    # Federated learning loop
    for epoch in range(num_epochs):
        print(f"\nGlobal Epoch {epoch + 1}/{num_epochs}")

        start_global_time = time.time()

        # Select clients using knapsack algorithm
        selected_clients_indices, _ = knapsack_with_features(clients, capacity)
        selected_clients = [clients[i] for i in selected_clients_indices]

        for client in selected_clients:
            print(f"\nClient {clients.index(client) + 1}/{len(clients)}")
            # Create a local model
            local_model = create_client_model()

            # Train the local model
            start_local_time = time.time()
            local_model.fit(X_train, Y_train, epochs=client_epochs, batch_size=batch_size, verbose=0)
            end_local_time = time.time()

            # Evaluate local model on client's data
            _, local_accuracy = local_model.evaluate(X_test, Y_test, verbose=0)
            local_loss = 1 - local_accuracy

            # Update knapsack score for the client
            knapsack_scores[clients.index(client)] = local_loss

            # Print results for each iteration
            print(f"Iteration {client_epochs}, Time: {end_local_time - start_local_time:.2f}s, Accuracy: {local_accuracy * 100:.2f}%, Loss: {local_loss:.4f}")

            # Aggregate models from selected clients into the global model (in this example, simply update the global model)
            global_model.set_weights(local_model.get_weights())

        end_global_time = time.time()
        global_accuracy = global_model.evaluate(X_test, Y_test)[1]
        global_loss = 1 - global_accuracy
        print(f"\nGlobal Epoch Time: {end_global_time - start_global_time:.2f}s, Global Accuracy: {global_accuracy * 100:.2f}%, Global Loss: {global_loss:.4f}")

    # Evaluate the final global model
    _, final_accuracy = global_model.evaluate(X_test, Y_test)
    print(f"\nFinal Global Model Accuracy: {final_accuracy * 100:.2f}%")

# Example:
clients = [
    {"Di": 6000, "Loss": 0.3372, "Delay": 1.421, "Energy": 58, "Reliability": 0.1210938634, "Fairness": 0.66},
    {"Di": 6000, "Loss": 0.6755, "Delay": 2, "Energy": 64, "Reliability": 0.04978706837, "Fairness": 0.83},
    {"Di": 6000, "Loss": 0.5286, "Delay": 2.16, "Energy": 47, "Reliability": 0.2493522088, "Fairness": 0.63},
    {"Di": 6000, "Loss": 0.4091, "Delay": 1.27, "Energy": 38, "Reliability": 1, "Fairness": 0.6},
    {"Di": 6000, "Loss": 0.3258, "Delay": 1.17, "Energy": 42, "Reliability": 1, "Fairness": 0.83},
    {"Di": 6000, "Loss": 0.4178, "Delay": 1.753, "Energy": 65, "Reliability": 0.1806214331, "Fairness": 1},
    {"Di": 6000, "Loss": 0.13236, "Delay": 1.8, "Energy": 36, "Reliability": 0.03567399335, "Fairness": 1},
    {"Di": 6000, "Loss": 0.2377, "Delay": 2.142, "Energy": 13, "Reliability": 1, "Fairness": 1},
    {"Di": 6000, "Loss": 0.323, "Delay": 2.52, "Energy": 16,"Reliability": 0.02811565975, "Fairness": 0.9333333333},
    {"Di": 6000, "Loss": 0.4094, "Delay": 1.8, "Energy": 96, "Reliability": 0.1888756028, "Fairness": 0.9333333333}
]
capacity = 200
NUMOFCLASSES = 10
LOSS = 'categorical_crossentropy'
OPTIMIZER = 'adam'
DROP_RATE = 0.25
EPOCHS = 30
CLIENT_EPOCHS = 3
BATCH_SIZE = 32

federated_learning_with_selection(clients, capacity, NUMOFCLIENTS, EPOCHS, CLIENT_EPOCHS, BATCH_SIZE)
